In [67]:
from compute import Var, Par
from datastructures.api import Model, adda, addf, addsolver, setsolvefor, evalexpr
from datastructures.execution import edges_from_components
from datastructures.graphutils import namefromsympy, dfs_tree, SOLVER, VAR, COMP, nested_sources, sources, edges_to_Ein_Eout, flat_graph_formulation, root_solver
from datastructures.nestedgraph import root_sources
from datastructures.workflow import get_f, order_from_tree, default_solver_options, mdao_workflow
from datastructures.workflow_mdao import mdao_workflow_with_args
from datastructures.operators import sort_scc, reorder_merge_solve
from datastructures.assembly import build_archi
from collections import OrderedDict
import matplotlib.pyplot as plt
from sympy import cos, pi, log, sqrt
import openmdao.api as om

# Model

## Variables

In [2]:
Df = Var('D_f', 2, unit='m')
Ds = Var('D_s', 1, unit='m')
Dd = Var('D_d', 1.25, unit='m')
tf = Var('t_f', 0.5, unit='m')
ts = Var('t_s', 2, unit='m')
td = Var('t_d', 0.5, unit='m')

## Parameters

In [3]:
rhow = Par(r'\rho_w', 1023.6, 'kg/m^3') # density of seawater [kg/m^3]
rho = Var(r'\rho', 700, 'kg/m^3')       # arbitrarily set
rhoh = Var(r'\rho_h', 2700, 'kg/m^3')   # arbitrarily set

In [79]:
model = Model()
m = model.root
idbyname = True

## Geometry

In [80]:
geometry = addsolver(m, name='geometry', idbyname=idbyname)

In [81]:
A_s = Var('A_{solar}', unit='m^2')
alpha = Par(r'\alpha', 0.5)
d = adda(geometry, 'd', alpha*Df)
adda(geometry, Df, (4*abs(A_s)/pi+d**2)**0.5)

D_f

## Hydrodynamics

In [82]:
hydro = addsolver(m, name='hydro', idbyname=idbyname)

In [83]:
hf = Var('h_f', 0.5, 'm')
g = Par('g', 9.81, 'm/s^2')

In [84]:
Vd = adda(hydro, 'Vd', pi/4*(Df**2*hf+Ds**2*ts+Dd**2*td))

In [85]:
FB = adda(hydro, 'F_B', rhow*Vd*g, unit='N')
FW = adda(hydro, 'F_W', FB)

In [86]:
#hf = adda(hydro, 'h_f', (4*Vd/pi-Ds**2*ts-Dd**2*td)/Df**2)

In [87]:
xd = td/2
xs = td+ts/2
xf1 = td+ts+hf/2
xf2 = td+ts+tf/2
totA1 = hf*Df+ts*Ds+td*Dd
totA2 = tf*Df+ts*Ds+td*Dd

In [88]:
KB = adda(hydro, 'K_B', (hf*Df*xf1+ts*Ds*xs+td*Dd*xd)/totA1) # z_B
KG = adda(hydro, 'K_G', (tf*Df*xf2+ts*Ds*xs+td*Dd*xd)/totA2) 
I = adda(m, 'I', pi/64*Df**4)
BM = adda(m, 'B_M', I/Vd)
#should be 3-5% according to http://web.mit.edu/13.012/www/handouts/Reading3.pdf
GM = adda(hydro, 'G_M', KB+BM-KG) 

In [89]:
mtot = Var('m_{platform}', unit='kg')

In [90]:
C33 = adda(hydro, 'C_{33}', pi*rhow*g/4*Df**2)
A33 = adda(hydro, 'A_{33}', 0.0525*pi*rhow*(Dd**3+Ds**3+Df**3))
omega0 = adda(hydro, r'\omega_0', (C33/(A33+mtot))**1/2, unit='rad/s', forceunit=True)

## Mass

In [91]:
mass = addsolver(m, name='mass', idbyname=idbyname)

In [92]:
adda(mass, mtot, FW/g, 'kg')
mbatt = Var('m_{batt}', unit='kg')
mprop = Par('m_{prop}', 50, 'kg') # mass of propulsion
mcomms = Par('m_{comms}', 50, 'kg') # mass of comms system
eta_solar = Par(r'\eta_{solar}', 10, 'kg/m^2')
msolar = adda(mass, 'm_{solar}', eta_solar*A_s)
mstruct = adda(mass, 'm_{struct}', mtot-mbatt+msolar+mcomms+mprop) 
adda(mass, td, 4/pi*(mstruct-Df**2*tf*rho-Ds**2*ts*rho)/(Dd**2*rhoh))

t_d

## Propulsion

In [93]:
prop = addsolver(m, name='prop', idbyname=idbyname)

In [94]:
S_wd = pi*((Dd/2)**2-(Ds/2)**2+(Dd/2)**2+2*(Dd/2)*td)
S_ws = 2*pi*(Ds/2)*ts
S_wf = pi*((Df/2)**2-(Ds/2)**2+2*(Df/2)*hf)
S_w = adda(prop, 'S_w', S_wd+S_ws+S_wf, 'm**2') # wetted surface
C_d = Par('C_d', 1) # estimate drag coefficient (a square flat plate at 90 deg to the flow is 1.17)
v = Var('v', 1, unit='m/s') # [m/s]
eta_m = Par(r'\eta_m', 0.75) # estimated, need to determine from motors?
# D = rhow*C_d*S_w*v**2*0.5
P_move = adda(prop, 'P_{move}', rhow*C_d*S_w*v**3/(2*eta_m), unit='W')

## Communications

In [95]:
comms = addsolver(m, name='comms', idbyname=idbyname)

In [96]:
db2dec = lambda x: 10**(x/10)
dec2db = lambda x: 10*log(abs(x), 10)

In [97]:
Pcomms = Var('P_{comms}', 0.5, 'W')

In [98]:
k = Par('k', 1.38065e-23, 'J/K')
c = Par('c', 3e8, 'm/s')
f = Par('f', 2.2, 'GHz')

In [99]:
Lambda = adda(comms, r'\lambda', c/f, unit='m')

In [100]:
eta_parab = Par(r'\eta_{parab}', 0.55)
theta_t = Par(r'\theta_t', 32)
error_t = Par('e_t', 27)

In [101]:
G_pt_db = adda(comms, 'G_{pt}', 44.3-dec2db(theta_t**2), forceunit=True)
L_pt_db = adda(comms, 'L_{pt_db}', -12*(error_t/theta_t)**2)
G_t = adda(comms, 'G_t', db2dec(G_pt_db+L_pt_db))

In [102]:
D_r = Par('D_r', 5.3, 'm')
G_r = adda(comms, 'G_r', eta_parab*(pi*D_r/Lambda)**2)

In [103]:
h = Par('h', 780, 'km')
Re = Par('Re', 6378, 'km')
S = adda(comms, 'S', sqrt(h*(h+2*Re)), unit='km', forceunit=True)

In [104]:
L_s = adda(comms, 'L_s', (Lambda/(4*pi*S))**2)

In [105]:
BER = Par('BER', 0.0001)
R = Var('R', 10, 'Mbit/s') # 1 per microseconds
T_s = Par('T_s', 135, 'K')
L_a = Par('L_a', db2dec(-0.3))
L_l = Par('L_l', db2dec(-1))
L_p = Par('L_p', db2dec(-0.1))
EN = adda(comms, 'EN', Pcomms*L_a*L_s*L_l*L_p*G_r*G_t/(k*T_s*R))

## Power

In [107]:
power = addsolver(m, name='power', idbyname=idbyname)

### Energy budget

In [108]:
# should t_move, t_comms, t_service and t_recharge add up to 24 hours?
t_mission = Par('t_{mission}', 24, 'hr')
t_comms = Par('t_{comms}', 1, 'hr')
t_move = Par('t_{move}', 1, 'hr')
t_service = Par('t_{service}', 12, 'hr')
t_recharge = Par('t_{recharge}', 12, 'hr')

In [109]:
P_hotel = Par('P_{hotel}', 50, 'W')
E_AUV = Par('E_{AUV}', 1.9, 'kW*hr') # AUV battery capacity (to be recharged), based on Bluefin-9
gamma = Par(r'\gamma', 1) # AUVs serviced per mission duration (aka in the time of t_mission)

In [110]:
E_move = adda(power, 'E_{move}', P_move*t_move, unit='kW*hr')
E_hotel = adda(power, 'E_{hotel}', P_hotel*t_mission, unit='kW*hr')
E_comms = adda(power, 'E_{comms}', Pcomms*t_comms, unit='W*hr')
E_service = adda(power, 'E_{service}', E_AUV * gamma, unit='kW*hr')
P_service = adda(power, 'P_{service}', E_service/t_service, unit='W')

In [111]:
E_required = adda(power, 'E_{required}', E_hotel+E_move+E_service+E_comms, unit='kW*hr')

### Solar panel sizing

In [113]:
E_recharge = adda(power, 'E_{recharge}', E_required, unit='kW*hr')
P_recharge = adda(power, 'P_{recharge}', E_recharge / t_recharge, unit='W')

In [114]:
eta_s = Par(r'\eta_s', 0.27)
phi_s = Par(r'\phi_s', 800, 'W/m**2')
theta_bar = Par(r'\theta', 55, 'deg')
Ideg = Par('I_{deg}', 0.9)
ddeg = Par('d_{deg}', 0.005)
Lsolar = Par('L_{solar}', 10) #lifetime in years, but units act weirdly in powers

In [115]:
adda(power, A_s, P_recharge / (eta_s * phi_s * cos(theta_bar) * Ideg * (1-ddeg)**Lsolar), unit='m^2', forceunit=True)

A_{solar}

### Battery sizing

In [116]:
mu_batt = Par(r'\mu_{battery}', 200, 'W*hr/kg') # Lithium ion
DOD = Par('DOD', 0.7)
eta_batt = Par(r'\eta_{battery}', 0.85) # transmission efficiency
nu_batt = Par(r'\nu_{battery}', 450, 'kW*hr/(m**3)')
N = Par('N', 1)
C = adda(m, 'C', E_required/(DOD*N*eta_batt), unit='kW*hr')
V_batt = adda(m, 'V_{batt}', C/nu_batt, unit='m**3')
m_batt_zero = Par('m_{batt_zero}', 5, 'kg')
adda(power, mbatt, C/mu_batt + m_batt_zero, unit='kg') # was already defined in the beginning

m_{batt}